In [1]:
%%time

#Modded from qutip to allow parallelism
import numpy as np
import scipy.linalg as la
from numpy import angle, pi
from qutip import Qobj, propagator

def floquet_modes_mod(H, T, args=None, parallel=False, sort=False, U=None):
    """
    Calculate the initial Floquet modes Phi_alpha(0) for a driven system with
    period T.

    Returns a list of :class:`qutip.qobj` instances representing the Floquet
    modes and a list of corresponding quasienergies, sorted by increasing
    quasienergy in the interval [-pi/T, pi/T]. The optional parameter `sort`
    decides if the output is to be sorted in increasing quasienergies or not.

    Parameters
    ----------

    H : :class:`qutip.qobj`
        system Hamiltonian, time-dependent with period `T`

    args : dictionary
        dictionary with variables required to evaluate H

    T : float
        The period of the time-dependence of the hamiltonian. The default value
        'None' indicates that the 'tlist' spans a single period of the driving.

    U : :class:`qutip.qobj`
        The propagator for the time-dependent Hamiltonian with period `T`.
        If U is `None` (default), it will be calculated from the Hamiltonian
        `H` using :func:`qutip.propagator.propagator`.

    Returns
    -------

    output : list of kets, list of quasi energies

        Two lists: the Floquet modes as kets and the quasi energies.

    """
    if 'opts' in args:
        options = args['opts']
    else:
        options = Options()
        options.rhs_reuse = True
        rhs_clear() 
    
    if U is None:
        # get the unitary propagator
        U = propagator(H, T, [], args, parallel=parallel, progressbar=True, options=options)
    
    # find the eigenstates for the propagator
    evals, evecs = la.eig(U.full())

    eargs = angle(evals)

    # make sure that the phase is in the interval [-pi, pi], so that
    # the quasi energy is in the interval [-pi/T, pi/T] where T is the
    # period of the driving.  eargs += (eargs <= -2*pi) * (2*pi) +
    # (eargs > 0) * (-2*pi)
    eargs += (eargs <= -pi) * (2 * pi) + (eargs > pi) * (-2 * pi)
    e_quasi = -eargs / T

    # sort by the quasi energy
    if sort:
        order = np.argsort(-e_quasi)
    else:
        order = list(range(len(evals)))

    # prepare a list of kets for the floquet states
    new_dims = [U.dims[0], [1] * len(U.dims[0])]
    new_shape = [U.shape[0], 1]
    kets_order = [Qobj(np.matrix(evecs[:, o]).T,
                       dims=new_dims, shape=new_shape) for o in order]

    return kets_order, e_quasi[order]

CPU times: user 1.42 s, sys: 3.5 s, total: 4.91 s
Wall time: 1.9 s


In [2]:
import numpy as np
import cupy as cp
from qutip import Qobj, propagator

from qutip import basis, jmat

def drive(t, args):
    w = args['omega']
    h = args['h']
    h0 = args['h0']
    return h * np.sin(w*t) + h0

def drive_gpu(t, args):
    w = args['omega']
    h = args['h']
    h0 = args['h0']
    return h * cp.sin(w*t) + h0

def get_hamiltonians(N):
    sx,sy,sz = jmat(N,"x"),jmat(N,"y"),jmat(N,"z")
    kn =  2.0/(N-1)                                      # kacNorm
    H0 = kn * sz **2 
    H1 = 2 * sx
    return H0,H1 #Hamiltonian per particle

def propagator_evolve(args):
    N = args['N']
    T = 2 * np.pi/args['omega']
    opts = args['opts']
    H0, H1 = get_hamiltonians(N)
    H = [H0,[H1,drive]]
    U = propagator(H, T, [], args, parallel=True, progressbar=False, options=opts)
    U_gpu = cp.array(U.full())
    return U_gpu

def floq_evolv(args):
    N = args['N']
    T = 2 * np.pi/args['omega']
    opts = args['opts']
    H0, H1 = get_hamiltonians(N)
    H = [H0,[H1,drive]]
    f_states, _ = floquet_modes_mod(H, T, args=args)
    return f_states

def observables(params, nperiods = 4000):
    umat_gpu = propagator_evolve(params)
    N = params['N']

    H0, H1 = get_hamiltonians(N)
    #t=0 Hamiltonian
    #Ht0 = H0 + drive(0, params) * H1
    #en, psi0 = Ht0.groundstate()

    #Typical state is fully spin polarized
    sx = jmat(N,"x")
    en, psi0 = sx.groundstate()
    sx_gpu = cp.array(sx.full())

    psi0_gpu = cp.array(psi0)

    H0_gpu, H1_gpu = cp.array(H0.full()),cp.array(H1.full()) 

    w = params['omega']
    T = 2 * cp.pi/omega
    hbar_data = cp.zeros(nperiods+1, dtype=cp.complex128)
    hvar_data = cp.zeros_like(hbar_data)
    
    
    sxavg_data = cp.zeros_like(hbar_data)
    sxvar_data = cp.zeros_like(hbar_data)
    norm = cp.zeros_like(hbar_data)

    hbar_data[0] = en/N
    sxavg = cp.conjugate(psi0_gpu).T @ sx_gpu @ psi0_gpu
    sxsq = cp.conjugate(psi0_gpu).T @ sx_gpu @ sx_gpu @ psi0_gpu
    sxavg_data[0] = sxavg/N
    sxvar_data[0] = (sxsq - (sxavg * sxavg))
    
    norm[0]= cp.linalg.norm(psi0_gpu)

    un = cp.eye(2*N+1)

    for n in cp.arange(1, nperiods+1):
        un = un @ umat_gpu
        psiT_gpu = un @ psi0_gpu
        HT_gpu = H0_gpu + drive_gpu(n*T, params) * H1_gpu
        hbar_gpu = cp.conjugate(psiT_gpu).T @ HT_gpu @ psiT_gpu
        hsqbar_gpu = cp.conjugate(psiT_gpu).T @ HT_gpu @ HT_gpu @ psiT_gpu
        hvar_gpu = hsqbar_gpu - (hbar_gpu * hbar_gpu)
        hbar_data[n] = hbar_gpu/N
        hvar_data[n] = hvar_gpu/(N*N)
        sxavg = cp.conjugate(psiT_gpu).T @ sx_gpu @ psiT_gpu
        sxsq = cp.conjugate(psiT_gpu).T @ sx_gpu @ sx_gpu @ psiT_gpu
        sxavg_data[n] = sxavg/N
        sxvar_data[n] = (sxsq - (sxavg * sxavg))/(N*N)
        norm[n] = cp.linalg.norm(psiT_gpu)
    
    obs = {"eavg":hbar_data.get(), "evar":hvar_data.get(),\
           "sxavg":sxavg_data.get(), "sxvar":sxvar_data.get(),"norm":norm.get()}
    
    return obs

def get_ipr(params):
    f_states = floq_evolv(params)
    sx = jmat(N,"x")
    en, st = sx.eigenstates()
    ipr = np.array([np.sum([np.abs(state.overlap(sx_ev))**4 for sx_ev in st]) for state in f_states])
    return ipr

print("Definitions complete!")

Definitions complete!


In [3]:
%%time
#Set parallelization here
nprocs = 32
import qutip
qutip.settings.num_cpus=nprocs

from qutip import Qobj, jmat, Options
import numpy as np
from tqdm import tqdm
from scipy.special import j0, jn_zeros

root_choice = 1
N = 100
h0 = np.pi/2
nperiods = 12000
#Where J_0 is maximum
freezing_pts = jn_zeros(1, 3)

o1 = np.linspace(1.29, 1.58, 5)
o2 = np.linspace(1.6, 2.5, 5)
o3 = np.linspace(2.8, 4.38, 5)
o4 = np.linspace(8, 9.4, 5)
o5 = np.linspace(9.5, 20.0, 5)


omega_vals = np.concatenate((o1, o2, o3, o4, o5))
n_omegas = len(omega_vals)

print("running for N = ", N, "with ",nprocs," processors, chunksize ", n_omegas)

#Qutip Solver Options
opts = Options(nsteps=1e6, num_cpus=nprocs, openmp_threads=1, atol=1e-12, rtol=1e-14)
fulldata = {}
for i,omega in enumerate(omega_vals):
    hroots = freezing_pts * omega/4
    h_frz = hroots[root_choice]
    params_frz = {'h0':h0, 'h':h_frz, 'omega':omega, 'N':N, 'opts':opts}
    obsdata_frz = observables(params_frz, nperiods=nperiods)
    obsdata_frz['ipr'] = get_ipr(params_frz)
    obsdata_frz['metadata'] = {'omega':omega,'h':h_frz}
    fulldata[str(i)] = obsdata_frz

running for N =  100 with  32  processors, chunksize  10
CPU times: user 7min 56s, sys: 18 s, total: 8min 14s
Wall time: 8min 17s


In [5]:
import hdfdict
import h5py
import numpy as np

fname = 'nfrz_multomega_20230412_1120.h5'
f = h5py.File(fname, 'w')
hdfdict.dump(fulldata, fname)
f.attrs['h0'] = h0
f.attrs['N'] = N
f.attrs['nperiods'] = nperiods

for key, value in opts.__dict__.items():
    if value is not None:
        f.attrs[str(key)] = value

f.close()